Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kanggiho","key":"da46870aaebfa67826f13d6e34899f5f"}'}

In [12]:
# @title DownLoad Data From Kaggle Dataset
# @markdown 1. Download API Credentials From Kaggle (file name is kaggle.json)

# @markdown 2. Setup the Colab Notebook and Upload API Credentials To Google Colab
!pip install -q kaggle
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
# @markdown 3. Download datasets
dataset_name = "jessicali9530/celeba-dataset" # @param {type:"string"}
!kaggle datasets download {dataset_name}
# @markdown 4. Unzip what you downloaded

# @markdown [reference material](https://deepnote.com/app/lulus/Load-dataset-from-kaggle-to-google-colab-411fca3f-ee05-45fd-b7e4-a28990260184)







Saving kaggle.json to kaggle.json
kaggle.json
100% 1.32G/1.33G [00:12<00:00, 188MB/s]
100% 1.33G/1.33G [00:12<00:00, 118MB/s]


In [13]:
!unzip -q celeba-dataset.zip

## 데이터 다운로드( URL = https://www.robots.ox.ac.uk/~vgg/data/pets/)

In [78]:
# %%writefile utils.py
import requests
import tarfile
from tqdm import tqdm
from pathlib import Path
from urllib.parse import urlparse
import os
 # https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
 # https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
def download_data(
    url:str,
    remove:bool = False
):
  # parser = urlparse(url)
  # url = parser.geturl()

  response = requests.get(url , stream=True)
  total_size = int(response.headers.get("content-length",0))
  block_size = 1024
  # tar_file = Path(__file__).parent / "download.tar"
  tar_file = Path.cwd().resolve() / "download.tar.gz"
  print("압축 파일 다운로드 시작")
  progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True)
  with open(tar_file , "wb") as file:
    for data in response.iter_content(block_size):
      progress_bar.update(len(data))
      file.write(data)
  progress_bar.close()
  print("\n압축 파일 다운로드 완료")

  print("압축 파일 풀기 시작")
  with open(tar_file , "rb") as file:
    with tarfile.open(fileobj=file,mode="r:gz" ) as tar:
      total_files = len(tar.getnames())
      progress_bar = tqdm(total=total_files, unit="files", unit_scale=True)
      for tarinfo in tar:
        progress_bar.update(1)
        tar.extract(tarinfo)
      progress_bar.close()
  print("압축 파일 압축완료 ")
  if remove:
    os.remove(tar_file)

download_data("https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz" , remove=True)



Writing utils.py


In [79]:
# %%writefile dataset.py
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from pathlib import Path
from PIL import Image
import torch

class OxfordPets(Dataset):
  def __init__(self ,
               data_path : str,
               transform = None,
               split :str = "train",
               **kwargs):
    self.data_path = Path(data_path)
    self.transform = transform
    imges = [img for img in self.data_path.iterdir() if img.suffix ==".jpg"]
    self.imges = imges[:int(len(imges)*0.75)] if split =="train" else imges[int(len(imges)*0.75):]
    # self.imges = sorted(imges , key = lambda x : (x.stem , int(x.stem.split("_")[-1])))
    # imges = sorted([img for img in self.data_path.iterdir() if img.suffix ==".jpg"] , key = lambda x : (x.stem , int(x.stem.split("_")[-1])))
    self.labels = set(["_".join(i.stem.split("_")[:-1]) for i in self.imges])
    self.class_to_idx = {c : idx for idx, c in enumerate(self.labels)}
    self.idx_to_class = {v:k for k , v  in self.class_to_idx.items()}
  def __len__(self):
    return len(self.imges)
  def __getitem__(self, idx) :
    file_name = self.imges[idx]
    label = "_".join(file_name.stem.split("_")[:-1])
    img = Image.open(file_name)
    img = img.convert("RGB")
    label = torch.tensor(self.class_to_idx[label], dtype= torch.long)
    if self.transform is not None:
      img = self.transform(img)

    return img , label

class Dataset():
  def __init__(self ,
               data_path : str,
               batch_size:int ,
               img_size : int,
               **kwargs ):
    self.data_path = data_path
    self.batch_size = batch_size
    self.img_size = img_size
  def setup(self):
    transform = transforms.Compose([
          transforms.Resize((self.img_size,self.img_size)),
          transforms.ToTensor(),
          transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
        ])
    self.train_dataset = OxfordPets(self.data_path , transform=transform , split="train")
    self.val_dataset = OxfordPets(self.data_path , transform=transform , split="val")

  def train_dataloader(self) -> DataLoader:
    return DataLoader(self.train_dataset , batch_size=self.batch_size , shuffle=True)
  def val_dataloader(self) -> DataLoader:
    return DataLoader(self.val_dataset , batch_size=self.batch_size , shuffle=True)


Writing dataset.py


In [77]:
# %%writefile model.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import List , Any
from abc import abstractmethod

class BaseVAE(nn.Module):
  def __init__(self) ->None:
    super().__init__()
  def encode(self , input:torch.Tensor) -> List[torch.Tensor]:
    raise NotImplementedError
  def decode(self , input:torch.Tensor) -> List[torch.Tensor]:
    raise NotImplementedError
  def sample(self , batch_size:int , device:int , **kwargs) ->torch.Tensor:
    raise NotImplementedError
  def generate(self , x:torch.Tensor , **kwargs) -> torch.Tensor:
    raise NotImplementedError
  @abstractmethod
  def forward(self ,*inputs:torch.Tensor)->torch.Tensor:
    pass
  @abstractmethod
  def loss_function(self ,*inputs:Any , **kwargs)->torch.Tensor:
    pass

class VAE(BaseVAE):
  def __init__(self ,
               in_channels :int,
               latent_dim : int,
               hidden_dims : List | None=None,
               **kwargs) -> None:
      super(VAE, self).__init__()
      self.latent_dim = latent_dim
      modules = []
      if hidden_dims is None:
        hidden_dims = [32,64,128,256,512]
      # ----------
      # encoder
      # ----------
      for h_dim in hidden_dims:
        modules.append(
            nn.Sequential(
                nn.Conv2d(in_channels , h_dim , 3,2,1),
                nn.BatchNorm2d(h_dim),
                nn.LeakyReLU(),
            )
        )
        in_channels = h_dim

      self.encoder = nn.Sequential(*modules)
      self.fc_mu = nn.Linear(hidden_dims[-1]*4 , latent_dim)
      self.fc_logvar = nn.Linear(hidden_dims[-1]*4 , latent_dim)

      # ----------
      # decoder
      # ----------

      modules = []
      self.decoder_input = nn.Linear(latent_dim , hidden_dims[-1]*4)

      hidden_dims = hidden_dims[::-1]
      for idx in range(len(hidden_dims)-1):
        modules.append(
            nn.Sequential(
                nn.ConvTranspose2d(hidden_dims[idx] , hidden_dims[idx+1],3,2,1,output_padding=1),
                nn.BatchNorm2d(hidden_dims[idx+1]),
                nn.LeakyReLU(),
            )
        )
      self.decoder = nn.Sequential(*modules)

      self.final_layer = nn.Sequential(
          nn.ConvTranspose2d(hidden_dims[-1] , hidden_dims[-1] , 3,2,1,output_padding=1),
          nn.BatchNorm2d(hidden_dims[-1]),
          nn.LeakyReLU(),
          nn.Conv2d(hidden_dims[-1] , out_channels=3 , kernel_size=3,stride=1,padding=1),
          nn.Tanh()
      )

  def encode(self , input:torch.Tensor) -> List[torch.Tensor]:
    """
    encode the input tensor into latent codes
    input : (B,C,H,W)
    """
    assert input.shape[1] == 3 , f"{input.shape[1]} , {input.shape}"
    output = self.encoder(input) # (B,C,H,W) => (B,last_hidden_dim , 2,2)
    assert output.shape[1] == 512 and output.shape[2] == 2 , f"{output.shape[1]} , {output.shape}"
    output = torch.flatten(output , start_dim=1) # (B,last_hidden_dim , 2,2) => (B,last_hidden_dim*4)
    assert output.shape[1] == 512*4 , f"{output.shape[1]} , {output.shape}"
    mu = self.fc_mu(output)  # (B,last_hidden_dim*4) => (B,last_hidden_dim)
    log_var = self.fc_logvar(output) # (B,last_hidden_dim*4) => (B,last_hidden_dim)
    assert mu.shape == log_var.shape == (input.shape[0] , self.latent_dim) , f"{mu.shape} , {log_var.shape} , {input.shape} , {output.shape}"
    return [mu,log_var]

  def decode(self, z):
    """
    decode the latent codes into images
    input : (B,latent_dim)
    output : (B,C,H,W)
    """
    z = self.decoder_input(z) # (B,latent_dim) => (B,last_hidden_dim*4)
    z = z.view(-1,512,2,2) # (B,last_hidden_dim*4) => (B,512,2,2)
    output = self.decoder(z) # (B,512,2,2) => (B,32,H//2,W//2)
    output = self.final_layer(output) #  (B,32,H//2,W//2) => (B,3,H,W)
    return output

  def reparameterize(self , mu:torch.Tensor , logvar:torch.Tensor) -> torch.Tensor:
    """
    mu = (B,latent_dim)
    logvar = (B,latent_dim)
    output : z =(B,latent_dim)
    """
    noise = torch.randn(mu.size() , device = mu.device)
    assert noise.shape == mu.shape , f"{noise.shape} , {mu.shape}"
    std = torch.exp(logvar) **0.5 # => variance = e^log_var , std = variance**0.5
    assert std.shape == mu.shape , f"{std.shape} , {mu.shape}"
    z = mu+(std*noise)
    assert z.shape == mu.shape , f"{z.shape} , {mu.shape}"
    return z

  def forward(self , input:torch.Tensor) -> List[torch.Tensor]:
    mu,log_var = self.encode(input)
    z = self.reparameterize(mu,log_var)
    return [self.decode(z) , input , mu, log_var]

  def loss_function(self, *args , **kwargs ):
    """
    Calulate VAE loss
    """
    recons = args[0] # generated_image
    input = args[1] # input_image
    mu = args[2] # mu
    log_var = args[3] # log_var

    KLD_weight = kwargs["M_N"]
    recons_loss = F.mse_loss(recons , input)
    KLD_loss = torch.mean(-0.5*torch.sum(1 + log_var - mu**2 - torch.exp(log_var),dim=1),dim=0)
    loss = recons_loss + KLD_weight*KLD_loss
    return {"loss":loss , "Reconstruction_Loss" : recons_loss.item() , "KLD":-KLD_loss.item()}

  def sample(self ,
            n_samples:int,
            device,
            **kwargs
            )->torch.Tensor:
    """
    sample from the latent space and return image
    """
    z = torch.randn(n_samples,self.latent_dim)
    z = z.to(device)
    return self.decode(z)

  def generate(self , x:torch.Tensor , **kwargs) -> torch.Tensor:
    """
    Given an input image x , and then return generate image

    """
    return self.forward(x)[0]




Overwriting model.py


In [76]:
# %%writefile vae_test.py
import unittest
from model import VAE
import torch

class TestVAE(unittest.TestCase):
  def setUp(self):
    self.model = VAE(3,128)
    self.latent_dim = 128
    self.batch_size = 8
    self.channels = 3
    self.img_size = (self.batch_size , self.channels ,64,64)

  def test_encode(self):
    mu,logvar = self.model.encode(torch.randn(self.img_size))
    self.assertEqual(mu.shape ,(self.batch_size , self.latent_dim))
    self.assertEqual(logvar.shape , (self.batch_size , self.latent_dim))

  def test_decode(self):
    output = self.model.decode(torch.randn(self.batch_size,self.latent_dim))
    self.assertEqual(output.shape , (self.batch_size , 3, 64, 64))

  def test_reparam(self):
    z = self.model.reparameterize(torch.randn(self.batch_size,self.latent_dim),torch.randn(self.batch_size,self.latent_dim))
    self.assertEqual(z.shape , (self.batch_size , self.latent_dim))

  def test_forward(self):
    x = torch.randn(self.img_size)
    output,input , mu,log_var = self.model(x)
    self.assertEqual(output.shape , x.shape)
    self.assertEqual(input.shape , x.shape)
    self.assertEqual(mu.shape, (self.batch_size , self.latent_dim))
    self.assertEqual(log_var.shape, (self.batch_size , self.latent_dim))

  def test_loss(self):
    x = torch.randn(self.img_size)
    result = self.model(x)
    loss = self.model.loss_function(*result , M_N = 0.005)
    print(loss)

if __name__ =="__main__":
  unittest.main()




Writing vae_test.py


In [72]:
## Train
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import save_image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VAE(3,128).to(device)
dataset = Dataset("/content/images" ,  64 , 64 )
dataset.setup()
train_dataloder = dataset.train_dataloader()
val_dataloder = dataset.val_dataloader()
optimizer = optim.Adam(model.parameters() , lr = 0.005)
epochs = 100
for epoch in range(epochs):
  with tqdm(train_dataloder,total=len(train_dataloder) ,desc="training") as pbar:
    for img,_ in train_dataloder:
      img = img.to(device)
      # label = label.to(device)
      result = model(img)
      losses = model.loss_function(*result , M_N = 0.00025)
      loss = losses["loss"]
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      pbar.set_postfix(loss=loss.item())
      pbar.update(1)
    print(
        f"[Epoch {epoch+1}/{epochs}] "
        f"[loss : {loss.item():.4f}] "
        f"[RC loss : {losses['Reconstruction_Loss']:.4f}] "
        f"[KL loss : {losses['KLD']:.4f}] "
    )
  with torch.no_grad():
    test_img , _ = next(iter(val_dataloder))
    output = model.generate(test_img.to(device))
    if not Path("output").exists():
      Path(r"output").mkdir(parents=True, exist_ok=True)

    save_image(torch.cat((test_img[:25].data , output[:25].cpu().data ) , axis = 0)  ,
               f"output/Reconstruction_{epoch}.png" , nrow=5, normalize = True)

    samples = model.sample(25 ,device)
    save_image(samples , f"output/samples_{epoch}.png" , nrow=5, normalize = True)










training: 100%|██████████| 87/87 [00:23<00:00,  3.66it/s, loss=0.177]


[Epoch 1/100] [loss : 0.1765] [RC loss : 0.1606] [KL loss : -63.5966] 


training: 100%|██████████| 87/87 [00:25<00:00,  3.38it/s, loss=0.139]


[Epoch 2/100] [loss : 0.1390] [RC loss : 0.1237] [KL loss : -61.1362] 


training: 100%|██████████| 87/87 [00:25<00:00,  3.38it/s, loss=0.14]


...

[Epoch 98/100] [loss : 0.0663] [RC loss : 0.0483] [KL loss : -72.3542] 


training: 100%|██████████| 87/87 [00:25<00:00,  3.37it/s, loss=0.0704]


[Epoch 99/100] [loss : 0.0704] [RC loss : 0.0526] [KL loss : -71.0697] 


training: 100%|██████████| 87/87 [00:25<00:00,  3.38it/s, loss=0.0619]


[Epoch 100/100] [loss : 0.0619] [RC loss : 0.0439] [KL loss : -71.8709] 


In [66]:
a ,b = next(iter(val_dataloder))
b

tensor([ 7,  8, 20,  1, 33,  8, 25, 26, 16, 35,  9,  9, 16, 18, 24, 36, 32,  2,
         2,  3, 20, 32, 32,  5, 27, 26, 18, 12, 10, 12, 31, 27, 13, 31, 17, 21,
        31, 14,  6,  4, 27,  9,  7, 13, 34, 10, 22,  2, 26, 34, 21, 20, 28, 19,
        30, 18, 28,  2,  9, 26, 33,  2, 16, 12])

Mounted at /content/drive
